In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Checks Python Version
!python --version

Python 3.6.9


# Imports

In [ ]:
# This cell allows us to import local modules from google drive
import sys
sys.path.append('/content/drive/Shared drives/Capstone Summer 2020/Collab Files/Scripts')

# LOCAL MODULES
import dataset as D
import transforms as T

# "UNet++" Class Section Imports
import torch
import torch.nn as nn

# "Plotting" Section Imports
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce

# "Training" Class Section Imports
from collections import defaultdict
import os
import copy
import time
import torch
import torch.nn.functional as F
from tqdm.notebook import tqdm 
from torch.utils.tensorboard import SummaryWriter
import logging

# "Dataset Processing" Section Imports
from torch.utils.data import Dataset as BaseDataset
from torchvision import transforms
from torch import optim
from torch.utils.data import DataLoader
import torchvision

# Displaying Masks
from PIL import Image
from torchvision.transforms import functional as FF

%load_ext tensorboard


# Data Class

In [ ]:
# preprocess the input image
def crop_image_with_percentage(img):
    wid, hei = img.size
    
    # (x1, x2) = 0%, 56.5%
    start_width_percentage = 0
    end_width_percentage = 0.56497175141
    # (y11, y2) = 17%, 75.3%
    start_height_percentage = 0.17023346303
    end_height_percentage = 0.75389105058
    
    new_pts = (round(start_width_percentage*wid), round(start_height_percentage*hei), round(end_width_percentage*wid), round(end_height_percentage*hei))
    return img.crop(new_pts)

def fix_mask(mask):
    mask[mask == 128] = 1
    mask[mask == 255] = 2 
    return mask

class cropped_Dataset(BaseDataset):
  def __init__(self, root_dir):
    self.root_dir = root_dir
    self.images_dir = os.path.join(root_dir, 'Images')
    self.masks_dir = os.path.join(root_dir, 'Masks')
    self.ids = os.listdir(self.images_dir)
    self.maskids = os.listdir(self.masks_dir)

  def __getitem__(self, idx):
    #load images and masks
    img_path = os.path.join(self.images_dir, self.ids[idx])
    mask_path = os.path.join(self.masks_dir, self.maskids[idx])
    original_image = Image.open(img_path).convert("RGB")
    mask = Image.open(mask_path)
    label = 0 if self.ids[idx][0] == 'n' else 1

    cropped_img =  crop_image_with_percentage(original_image)
    mask = crop_image_with_percentage(mask)

    # transform
    cropped_img = cropped_img.resize((256, 256))
    cropped_img = FF.to_tensor(np.array(cropped_img))
    mask = mask.resize((256,256), resample = Image.NEAREST)

    # Fixing mask values 
    mask = np.array(mask)
    mask = np.expand_dims(mask, axis=0)

    mask =  torch.from_numpy(mask)
    mask = fix_mask(mask)
    sample = {'image': cropped_img, 'mask': mask}
    
    return sample
  
  def __len__(self):
    return len(self.ids)

## Dataset

In [ ]:
train_path_old = 'drive/Shared drives/Capstone Summer 2020/Data/REFUGE-Training400/'
validation_path_old ='drive/Shared drives/Capstone Summer 2020/Data/REFUGE-Validation400'

test_path = 'drive/Shared drives/Capstone Summer 2020/Data/Testing/'
train_path = 'drive/Shared drives/Capstone Summer 2020/Data/Training'
val_path ='drive/Shared drives/Capstone Summer 2020/Data/Validation'

root_path = 'drive/Shared drives/Capstone Summer 2020/Data/'

model_path = 'drive/Shared drives/Capstone Summer 2020/Models'


validation_image_path = 'drive/Shared drives/Capstone Summer 2020/Data/Validation/Images/'
validation_mask_path = 'drive/Shared drives/Capstone Summer 2020/Data/Validation/Masks/'

#--------- Tensor Board Directory ---------
logs_base_dir = 'drive/Shared drives/Capstone Summer 2020/Models/UNet++/runs'

In [ ]:
datasets = {x: cropped_Dataset(os.path.join(root_path, x)) for x in ['Training', 'Validation']}
dataloaders = {x: DataLoader(datasets[x], batch_size = 5, shuffle=True, num_workers=4) for x in ['Training', 'Validation']}
dataset_sizes = {x: len(datasets[x]) for x in ['Training', 'Validation']}

# Loss Class

In [ ]:
class HybridLogisticDiceLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, prediction, target):
        N, C, W, H = prediction.size()
        spatial_dims = (2, 3)

        target = binarize_labels(target, C)

        overlap = torch.sum(prediction * target, dim=spatial_dims)
        total = torch.sum(target * target, dim=spatial_dims) + torch.sum(prediction * prediction, dim=spatial_dims)
        dice_coeffs = 2 * overlap / total

        # small eps to avoid blowing up if the model ever starts getting really confident and predicts a 0.0 score
        # This was happening around epoch 20 for NestedUNet
        eps = 1e-8
        logistic = torch.sum(target * (torch.log(prediction + eps) - eps), dim=spatial_dims) / (W * H)

        hybrid = -logistic - dice_coeffs + 1

        return torch.sum(torch.sum(hybrid, dim=1) / C) / N
       
def binarize_labels(x, num_classes):
    x_bin = torch.zeros_like(x).repeat(1, num_classes, 1, 1)
    for i in range(num_classes):
        x_bin[:, i:i + 1][(x == i)] = 1. 
    return x_bin

def one_hot(x):
    C = x.size()[1]
    x = nn.functional.one_hot(torch.argmax(x, dim=1), num_classes=C)
    return x.permute(0, 4, 1, 2, 3)
    
def dice_metric(prediction, target):
    N, C, W, H = prediction.size()
    spatial_dims = (2, 3)

    # sets the highest prediction to 1, all others to 0
    prediction = one_hot(prediction)
    target = binarize_labels(target, C)

    overlap = torch.sum(prediction * target, dim=spatial_dims)
    total = torch.sum(target, dim=spatial_dims) + torch.sum(prediction, dim=spatial_dims)

    dice_coeffs = 2 * overlap / total

    return dice_coeffs

# UNet++ Class

In [ ]:
class conv_block_nested(nn.Module):
    def __init__(self, in_ch, mid_ch, out_ch):
        super(conv_block_nested, self).__init__()
        self.activation = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_ch, mid_ch, kernel_size=3, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(mid_ch)
        self.conv2 = nn.Conv2d(mid_ch, out_ch, kernel_size=3, padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(out_ch)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.activation(x)

        x = self.conv2(x)
        x = self.bn2(x)
        output = self.activation(x)

        return output

class Nested_UNet(nn.Module):
    def __init__(self, in_ch=3, out_ch=1):
        super(Nested_UNet, self).__init__()

        n1 = 64
        filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv0_0 = conv_block_nested(in_ch, filters[0], filters[0])
        self.conv1_0 = conv_block_nested(filters[0], filters[1], filters[1])
        self.conv2_0 = conv_block_nested(filters[1], filters[2], filters[2])
        self.conv3_0 = conv_block_nested(filters[2], filters[3], filters[3])
        self.conv4_0 = conv_block_nested(filters[3], filters[4], filters[4])

        self.conv0_1 = conv_block_nested(filters[0] + filters[1], filters[0], filters[0])
        self.conv1_1 = conv_block_nested(filters[1] + filters[2], filters[1], filters[1])
        self.conv2_1 = conv_block_nested(filters[2] + filters[3], filters[2], filters[2])
        self.conv3_1 = conv_block_nested(filters[3] + filters[4], filters[3], filters[3])

        self.conv0_2 = conv_block_nested(filters[0]*2 + filters[1], filters[0], filters[0])
        self.conv1_2 = conv_block_nested(filters[1]*2 + filters[2], filters[1], filters[1])
        self.conv2_2 = conv_block_nested(filters[2]*2 + filters[3], filters[2], filters[2])

        self.conv0_3 = conv_block_nested(filters[0]*3 + filters[1], filters[0], filters[0])
        self.conv1_3 = conv_block_nested(filters[1]*3 + filters[2], filters[1], filters[1])

        self.conv0_4 = conv_block_nested(filters[0]*4 + filters[1], filters[0], filters[0])

        self.final = nn.Conv2d(filters[0], out_ch, kernel_size=1)

    def forward(self, x):
        x0_0 = self.conv0_0(x)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.Up(x1_0)], 1))

        x2_0 = self.conv2_0(self.pool(x1_0))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.Up(x2_0)], 1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.Up(x1_1)], 1))

        x3_0 = self.conv3_0(self.pool(x2_0))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.Up(x3_0)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.Up(x2_1)], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.Up(x1_2)], 1))

        x4_0 = self.conv4_0(self.pool(x3_0))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.Up(x4_0)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.Up(x3_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.Up(x2_2)], 1))
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.Up(x1_3)], 1))

        output = self.final(x0_4)

        return output

# Train Class

In [ ]:
def calculate_loss(pred, target, metrics):
    criterion = HybridLogisticDiceLoss()
    pred = torch.sigmoid(pred)
    loss = criterion(pred, target)
    
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)

    return loss

def compute_metrics(metrics, epoch_samples):
    computed_metrics = {}
    for k in metrics.keys():
        computed_metrics[k] = metrics[k] / epoch_samples
    return computed_metrics
    
def print_metrics(computed_metrics, phase):
    outputs = []
    for k in computed_metrics.keys():
        outputs.append("  {}:{:4f}".format(k, computed_metrics[k]))
    print("  {} -> {}".format(phase.ljust(5), " |".join(outputs)))

def load_model(model_dir, filename):
    model = torch.load(os.path.join(model_dir, filename))
    return model

class Train(object):
    def __init__(self, model, optimizer, scheduler):
        # 1) used to give access to methods and properties of a parent or sibling class
        # 2) uses GPU if available
        super().__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(self.device)
        self.optimizer = optimizer
        self.scheduler = scheduler

    def train_model(self, dataloaders, log_path_train, log_path_val, batch_size = 5, num_epochs = 25, best_loss = 1e10):
        n_train = 400
        n_val = 400

        writer = SummaryWriter(log_dir = log_path_train)
        writer2 = SummaryWriter(log_dir = log_path_val)
        
        global_step = 0

        start = time.time()
        best_model_weights = copy.deepcopy(self.model.state_dict())

        epochs_metrics = {'Training': [],'Validation': []}
        for epoch in range(num_epochs):
            print('Epoch {}/{}'.format(epoch + 1, num_epochs))
            print('-' * 10)

            epoch_start = time.time()
            # Training & Validation Phase for each epoch
            for phase in ['Training', 'Validation']:
                if phase == 'Training':
                    self.model.train() # Sets model to Training mode
                else:
                    self.model.eval()  # Sets model to Evaluate mode
                metrics = defaultdict(float)
                epoch_samples = 0

                for sample in iter(dataloaders[phase]):
                    inputs = sample['image'].to(self.device)
                    masks = sample['mask'].to(self.device)

                    # Zero the paramater gradients
                    self.optimizer.zero_grad()

                    # --FORWARD--
                    # tracks history if only in 'train'
                    with torch.set_grad_enabled(phase == 'Training'):
                        outputs = self.model(inputs)
                        loss = calculate_loss(outputs, masks, metrics)

                        # --BACKWARD--
                        if phase == 'Training':
                            loss.backward()
                            self.optimizer.step()
                    # Statistics
                    epoch_samples += inputs.size(0)
                    global_step += 1

                    if global_step % (n_train // (10 * batch_size)) == 0 and phase == 'Training':
                        comp_metrics = compute_metrics(metrics, epoch_samples)
                        # print_metrics(comp_metrics, phase)
                        epochs_metrics[phase].append(comp_metrics)
                        epoch_loss = metrics['loss'] / epoch_samples

                        writer.add_scalar("loss/train", comp_metrics['loss'], global_step)
                        
                    elif global_step % (n_val // (10 * batch_size)) == 0 and phase == 'Validation':
                        comp_metrics = compute_metrics(metrics, epoch_samples)
                        # print_metrics(comp_metrics, phase)
                        epochs_metrics[phase].append(comp_metrics)
                        epoch_loss = metrics['loss'] / epoch_samples

                        writer2.add_scalar("loss/val", comp_metrics['loss'], global_step)

                comp_metrics = compute_metrics(metrics, epoch_samples)
                print_metrics(comp_metrics, phase)
                epochs_metrics[phase].append(comp_metrics)
                epoch_loss = metrics['loss'] / epoch_samples

                if phase == 'Training':
                    self.scheduler.step()

                # deep copy the best model
                if phase == 'Validation' and epoch_loss < best_loss:
                    print("  SAVING BEST MODEL: Epoch loss {:4f} < Best loss {:4f}".format(epoch_loss, best_loss))
                    best_loss = epoch_loss
                    best_model_wts = copy.deepcopy(self.model.state_dict())

            time_epoch_elapsed = time.time() - epoch_start
            print('  {:.0f}m {:.0f}s'.format(time_epoch_elapsed // 60, time_epoch_elapsed % 60))
            print('\n')
        
        # Prints the Best Val Loss Value
        time_elapsed = time.time() - start
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('Best Validation Loss: {:4f}'.format(best_loss))

        # Load the best model weights
        self.model.load_state_dict(best_model_weights)

        return self.model

In [ ]:
# model = Nested_UNet(in_ch = 3, out_ch = 3)
# log_dir_train = 'drive/Shared drives/Capstone Summer 2020/Models/TEST_UNet++/test_run2/train'
# log_dir_val = 'drive/Shared drives/Capstone Summer 2020/Models/TEST_UNet++/test_run2/val'

# optimizer_func = optim.Adam(model.parameters(), lr = 1e-4)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_func, step_size = 10, gamma = 0.1)

# trainer = Train(model, optimizer = optimizer_func, scheduler = lr_scheduler)
# trained_model_UNetpp = trainer.train_model(dataloaders, log_path_train = log_dir_train, log_path_val = log_dir_val, batch_size = 5, num_epochs = 25)

In [ ]:
# %tensorboard --logdir 'drive/Shared drives/Capstone Summer 2020/Models/TEST_UNet++/test_run2'

In [ ]:
# torch.save(trained_model_UNetpp, os.path.join(model_path, 'UNet++_Test2.pt'))

## Model Names:
---
#### **Model Parameters:**
---
**MODEL NAME: UNet++_Test:** \\
> *Output Channels: 3* \\
> *Loss Function: Hybrid Logistic Dice Loss* \\
> *Learning Rate: 0.0001* \\
> *Step Size: 10* \\
> *Gamma: 0.1* \\
> *Epochs: 25* \\
> *BATCH SIZE: 3*
---
**MODEL NAME: UNet++_Test2:** \\
> *Output Channels: 3* \\
> *Loss Function: Hybrid Logistic Dice Loss* \\
> *Learning Rate: 0.0001* \\
> *Step Size: 10* \\
> *Gamma: 0.1* \\
> *Epochs: 25* \\
> *BATCH SIZE: 5*


# ONH Detection

In [ ]:
# import the necessary packages
from imutils import contours
from skimage import measure
import numpy as np
import argparse
import imutils
import cv2
from google.colab.patches import cv2_imshow
import PIL
import os

def cropONH(imageName):
	left_bias = 128
	output_dim = 512
	def calc_gt_bounds(msk_path):
		gt = PIL.Image.open(msk_path)
		mw, mh = gt.size

		gt = 255 - np.array(gt)
		gt_T = gt.T

		for i in range(gt.shape[0]):
			if (127 in gt[i]) and (0 in gt[i]):
				h1 = i
				break
		for i in range(gt.shape[0]):
			if (127 in gt[-i]) and (0 in gt[-i]):
				h2 = mh - i
				break
		for i in range(gt_T.shape[0]):
			if (127 in gt_T[i]) and (0 in gt_T[i]):
				w1 = i
				break
		for i in range(gt_T.shape[0]):
			if (127 in gt_T[-i]) and (0 in gt_T[-i]):
				w2 = mw - i
				break
		return h1, h2, w1, w2
	
	data_set = {"n":"Training","g":"Training", "V":"Validation", "T":"Testing"}
	
	folderName= data_set[imageName[0]]
	folder_path = "drive/Shared drives/Capstone Summer 2020/Data/Original/" + folderName 
	
	img_path = folder_path + "/Images/" + imageName + ".jpg"
	mask_path = folder_path + "/Masks/" + imageName + ".bmp"
	
	y1, y2, x1, x2 = calc_gt_bounds(mask_path)
	image = PIL.Image.open(img_path)
	
	im_w, im_h = image.size
	var = round(0.15 * im_w)
	curr_threshold = starting_threshold = 250

	while True:
		# load the image, convert it to grayscale, and blur it
		image = cv2.imread(img_path)
  
		gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
		blur = cv2.bilateralFilter(gray,9,75,75)
		median=cv2.medianBlur(gray,5)

		# threshold the image to reveal light regions in the blurred image
		thresh = cv2.threshold(median, curr_threshold, 255, cv2.THRESH_BINARY)[1]

		# perform a series of erosions and dilations to remove any small blobs of noise from the thresholded image
		thresh = cv2.erode(thresh, None, iterations=2)
		thresh = cv2.dilate(thresh, None, iterations=4)

		# perform a connected component analysis on the thresholded image, then initialize a mask to store only the "large" components
		labels = measure.label(thresh, connectivity=2, background=0)
		mask = np.zeros(thresh.shape, dtype="uint8")

		largest_blob = 0
		# loop over the unique components
		for label in np.unique(labels):
			# if this is the background label, ignore it
			if label == 0:
				continue
			# otherwise, construct the label mask and count the number of pixels 
			labelMask = np.zeros(thresh.shape, dtype="uint8")
			labelMask[labels == label] = 255
			numPixels = cv2.countNonZero(labelMask)

			# if the number of pixels in the component is sufficiently large, then add it to our mask of "large blobs"
			if numPixels > largest_blob:
				largest_blob = numPixels 
				mask = labelMask


		# find the contours in the mask, then sort them from left to right
		cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
		cnts = imutils.grab_contours(cnts)
	
		#If there is nothing found for the image
		if cnts == []:
			curr_threshold -= 10
			
			continue
		print("Decreased threshold by: {}".format(starting_threshold - curr_threshold))
		cnts = contours.sort_contours(cnts)[0]

		# loop over the contours
		for (i, c) in enumerate(cnts):
			(x, y, w, h) = cv2.boundingRect(c)

			center = (round(x+(w/2)), round(y+(h/2)))
			cv2.putText(image, "O", center, cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
   
			box_radius = output_dim//2
			tl_pt = [center[0]-box_radius-left_bias, center[1]-box_radius-left_bias]
			br_pt = [center[0]+box_radius, center[1]+box_radius]

			#Check if TL point is out of bounds
			if tl_pt[0] < 0:
				neg = tl_pt[0] * -1
				tl_pt[0] = 0
				br_pt[0] += neg

			if tl_pt[1] < 0:
				neg = tl_pt[1] * -1
				tl_pt[1] = 0
				br_pt[1] += neg

			#Check if BR point is out of bounds
			if br_pt[0] > im_w:
				pos = im_w-br_pt[0]
				br_pt[0] = im_w
				tl_pt[0] -= pos
			if br_pt[1] > im_h:
				pos = im_h-br_pt[1]
				br_pt[1] = im_h
				tl_pt[1] -= pos
	
			cv2.rectangle(image,tuple(tl_pt) , tuple(br_pt), (255,0,0), 8)
			break
		
		# Checks if Ground Truth Bounds are within Mask Bounds
		if not ((x1 >= tl_pt[0]) and (x2 <= br_pt[0]) and (y1 >= tl_pt[1]) and (y2 <= br_pt[1])):
			print("-"*50)
			print("GT Bounds aren't within Mask Bounds for the IMAGE: {}".format(imageName))
			print("-"*50)
   
		print("Wid: ", br_pt[0]-tl_pt[0], "\tHei:", br_pt[1]-tl_pt[1])
		
		# Opens a new image without the rectangles and stuff
		orig_image = cv2.imread(img_path)
		gt = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
		# Crops the 'image/ground truth' to the mask bounds and dsiplays the cropped 'image/ground truth'
		cropped_image = orig_image[tl_pt[1]:br_pt[1],tl_pt[0]:br_pt[0]]
		cropped_gt = gt[tl_pt[1]:br_pt[1],tl_pt[0]:br_pt[0]]
		
		# print(np.asarray(cropped_gt).shape)
		# c_img_path = "drive/Shared drives/Capstone Summer 2020/Data/Cropped_Training/C_Images/" + imageName + ".jpg"
		c_msk_path = "drive/Shared drives/Capstone Summer 2020/Data/Validation/Masks/" + imageName + ".bmp"

		# SAVES THE CROPPED IMAGES AND MASKS
		# cv2.imwrite(c_img_path, cropped_image)
		cv2.imwrite(c_msk_path, cropped_gt)
		# cv2_imshow(cropped_gt)
		break

***Training***
* All images are good

***Validation*** 
* All images are good

In [ ]:
# cropONH("n0001")

# folderPath = 'drive/Shared drives/Capstone Summer 2020/Data/Original/Training/Images/' 
# for file in os.listdir(folderPath):
#     fname = file.split(".")[0]
#     print(fname)
#     cropONH(fname)

folderPath = 'drive/Shared drives/Capstone Summer 2020/Data/Original/Validation/Images/' 
for file in os.listdir(folderPath):
    fname = file.split(".")[0]
    print(fname)
    cropONH(fname)


# img_dir = 'drive/Shared drives/Capstone Summer 2020/Data/Training/Images/'
# ext = '.jpg'
# for i in range(1, 361):
#     img_name = "n" + str(i).zfill(4)
#     if not os.path.exists(img_dir + img_name + ext):
#         continue
#     else:
#         print(img_name)
#         cropONH(img_name)

# for i in range(1, 41):
#     img_name = "g" + str(i).zfill(4)
#     if not os.path.exists(img_dir + img_name + ext):
#         continue
#     else:
#         print(img_name)
#         cropONH(img_name)

V0333
Decreased threshold by: 20
Wid:  640 	Hei: 640
V0125
Decreased threshold by: 70
Wid:  640 	Hei: 640
V0214
Decreased threshold by: 0
Wid:  640 	Hei: 640
V0286
Decreased threshold by: 10
Wid:  640 	Hei: 640
V0160
Decreased threshold by: 10
Wid:  640 	Hei: 640
V0010
Decreased threshold by: 10
Wid:  640 	Hei: 640
V0252
Decreased threshold by: 0
Wid:  640 	Hei: 640
V0319
Decreased threshold by: 0
Wid:  640 	Hei: 640
V0192
Decreased threshold by: 20
Wid:  640 	Hei: 640
V0065
Decreased threshold by: 30
Wid:  640 	Hei: 640
V0003
Decreased threshold by: 10
Wid:  640 	Hei: 640
V0381
Decreased threshold by: 0
Wid:  640 	Hei: 640
V0285
Decreased threshold by: 10
Wid:  640 	Hei: 640
V0318
Decreased threshold by: 0
Wid:  640 	Hei: 640
V0273
Decreased threshold by: 30
Wid:  640 	Hei: 640
V0143
Decreased threshold by: 10
Wid:  640 	Hei: 640
V0167
Decreased threshold by: 0
Wid:  640 	Hei: 640
V0077
Decreased threshold by: 10
Wid:  640 	Hei: 640
V0011
Decreased threshold by: 10
Wid:  640 	Hei: 640

In [ ]:
# img_dir = 'drive/Shared drives/Capstone Summer 2020/Data/Validation/Images/'
# ext = '.jpg'
# for i in range(1, 401):
#     img_name = "V" + str(i).zfill(4)
#     if not os.path.exists(img_dir + img_name + ext):
#         continue
#     else:
#         print(img_name)
#         cropONH(img_name)